In [ ]:
%pylab inline

import numpy as np
from sklearn.externals import joblib
import xarray as xr
from xnoah.data_matrix import unstack_cat, stack_cat
import pandas as pd

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA

In [ ]:
def mysel(x):
    return x.isel(z=slice(0, 30))

X = xr.open_dataset("X.nc")#.pipe(mysel)
Y = xr.open_dataset("Y.nc").pipe(mysel)

mu = X.mean(['time', 'x', 'y'])

In [ ]:
X_scale = (X-mu)/(mu+ 1e-6)

In [ ]:

def prepvar(X):
   return  stack_cat(X, "features", ['z']).stack(samples=['time', 'x', 'y']).transpose("samples", "features")


xmat = prepvar(X_scale)
ymat = prepvar(Y)
n, m = xmat.shape

In [ ]:
# mod = make_pipeline(PCA(n_components=20), LinearRegression())
mod = Ridge(0.1)

rand = np.random.choice(n, 10000 )
mod.fit(xmat[rand], ymat[rand])
mod.score(xmat[rand], ymat[rand])

In [ ]:
lrf = mod.predict(np.eye(m)) - mod.predict(0*np.eye(m))

row, col = xmat.indexes['features'], ymat.indexes['features']
lrf = pd.DataFrame(lrf, index=row, columns=col)

In [ ]:
plt.pcolormesh(lrf.loc["SL"]["Q1"].T)
plt.colorbar()

In [ ]:
plt.pcolormesh(lrf.loc["QT"]["Q1"].T)
plt.colorbar()

The linear response functions are much more reasonable when percent is used as a metric.

# turning w into a data matrix

In [ ]:
w.loc[idx.levels[1][0]]

In [ ]:
w = xr.open_dataarray("w.nc").to_dataframe('w').drop('time',1)
idx = xmat.indexes['features']

def f(i):
    if i < 0:
        return 1.0
    else:
        return float(w.loc[idx.levels[1][i]])

w_np = np.array([f(i) for i in idx.labels[1]])